In [2]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.image as mpimg
import random
from skimage.transform import resize
from sklearn.model_selection import train_test_split



Data preprossing


In [3]:
cat_dir="/kaggle/input/microsoft-catsvsdogs-dataset/PetImages/Cat"
dog_dir="/kaggle/input/microsoft-catsvsdogs-dataset/PetImages/Dog"

In [4]:
# Function to handle potential errors while loading images
def load_image(image_path):
    try:
        img = mpimg.imread(image_path)  
        return img
    except (FileNotFoundError, IOError) as e:
        print(f"Error loading image at {image_path}: {e}")
        return None  # Return None to handle missing images


In [5]:
def resize_image(img, target_size=(160, 144, 3)):
    resized_img = resize(img, target_size, order=1, anti_aliasing=True)
    return resized_img

In [6]:
def dir_imgs_to_nparr(directory_path,image_shape=(160, 144, 3)):
    
    X = np.empty((len(os.listdir(directory_path)), *image_shape), dtype=np.float32)  
    i = 0
    for filename in os.listdir(directory_path):
        if i==2000:
            break
        image_path = os.path.join(directory_path, filename)
        img =load_image(image_path)
        if img is not None:  
            img=resize_image(img)
            X[i]=img
            i+=1
   
    return X[0:i]  
    
    

In [7]:
X_cat= dir_imgs_to_nparr(cat_dir)
X_dog= dir_imgs_to_nparr(dog_dir)


/opt/conda/lib/python3.10/site-packages/PIL/TiffImagePlugin.py:864: UserWarning: Truncated File Read
  warnings.warn(str(msg))


In [9]:
X_cat.shape

(2000, 160, 144, 3)

In [10]:
def gen_XY(X_cat,X_dog):
    Y_cat=np.ones ((X_cat.shape[0],1), dtype=int)
    Y_dog=np.zeros((X_dog.shape[0],1))
    X=np.concatenate((X_cat,X_dog), axis=0)  
    Y=np.concatenate((Y_cat,Y_dog), axis=0)    
    
    combined_data = list(zip(X, X))
    random.shuffle(combined_data)

    X, Y = zip(*combined_data)

    X =np.array(X)
    Y =np.array(Y)
    return X,Y


In [11]:
X,Y=gen_XY(X_cat,X_dog)

In [12]:
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, random_state=42)


In [13]:
print("X_train shape:", X_train.shape)
print("X_val shape:", X_val.shape)
print("y_train shape:", y_train.shape)
print("y_val shape:", y_val.shape)

X_train shape: (3200, 160, 144, 3)
X_val shape: (800, 160, 144, 3)
y_train shape: (3200, 160, 144, 3)
y_val shape: (800, 160, 144, 3)


**Model**